In [26]:
import dask
import dask.array as da
import dask.dataframe as dd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import StandardScaler, LabelEncoder

In [27]:
data = dd.read_csv('../data/intervehicle.csv')

In [3]:
y_data = data.pop('Label')
x_data = data
print('No of rows in the dataset',len(x_data))
print('Dimensions of the x_data',len(x_data.columns))

No of rows in the dataset 12746440
Dimensions of the x_data 70


In [4]:
train_x,test_x,train_y,test_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0,shuffle=True)
print('Rows in Training data',len(train_y))
print('Rows in Testing data',len(test_y))

Rows in Training data 10196215
Rows in Testing data 2550225


In [5]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True)
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [6]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.fit(test_x)

In [7]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [8]:
train_x['Label'] = train_y
train = train_x

In [9]:
model = Sequential()
model.add(Dense(70,input_dim=70))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(70))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [23]:
model.compile(optimizer='adam',loss='binary_crossentropy')

In [24]:
def data_generator(dataset,fraction=0.001):
  while True:
    batch = dataset.sample(frac=fraction)
    Y = batch.pop('Label')
    X = batch
    yield (X.compute(), Y.compute())

In [ ]:
model.fit(data_generator(train),steps_per_epoch=1000,epochs=5)

In [ ]:
score, accuracy = model.evaluate()